# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [36]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews.tile_sources as gvts
import geoviews as gv


# Import API key
from api_keys import geoapify_key
print(geoapify_key)

TypeError: descriptor '__call__' for 'type' objects doesn't apply to a 'property' object

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,as sulayyil,20.4607,45.5779,37.14,11,100,1.30,SA,1722111784
1,1,tamanrasset,22.7850,5.5228,33.95,17,40,2.06,DZ,1722111784
2,2,puerto natales,-51.7236,-72.4875,0.25,86,75,5.14,CL,1722111685
3,3,santa ana,33.7456,-117.8678,33.22,57,20,4.63,US,1722111318
4,4,jumla,29.2747,82.1838,16.14,91,100,0.55,NP,1722111785


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
import geoviews.tile_sources as gvts
import geoviews as gv

# Define the tile source for the map background
tiles = gv.tile_sources.OSM
# tiles = gvts.OSM


# use hvplot.points to setup the map parameters 
points = city_data_df.hvplot.points('Lng', 'Lat', size='Humidity', scale= 1.0, geo=True, frame_width= 700, frame_height= 500, tiles=tiles,  
                                    title='City Humidity', color = "City")
# Display the map
points



# Display the map
# YOUR CODE HERE

TypeError: descriptor '__call__' for 'type' objects doesn't apply to a 'property' object

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] < 25) & 
                               (city_data_df['Max Temp'] > 15) & 
                               (city_data_df['Wind Speed'] > 10) & 
                               (city_data_df['Wind Speed'] < 20) ]

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE



# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
180,180,swift current,50.2834,-107.8014,24.45,27,0,10.29,CA,1722111728
334,334,hobyo,5.3505,48.5268,24.53,85,62,11.11,SO,1722111733
411,411,sola,-13.8833,167.5500,24.79,66,100,10.30,VU,1722111947
488,488,mugia,43.1041,-9.2179,21.76,86,100,11.27,ES,1722111728


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
180,swift current,CA,50.2834,-107.8014,27,
334,hobyo,SO,5.3505,48.5268,85,
411,sola,VU,-13.8833,167.5500,66,
488,mugia,ES,43.1041,-9.2179,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
# params = {
#     "categories": "accommodation.hotel",
#     "limit": 1,
#     "apiKey": "geoapify_key",}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # params["filter"] = f"circle:{longitude},{latitude},{radius}"
    # params["bias"] = f"proximity:{longitude},{latitude}"
    params = {
    "categories": "accommodation.hotel",
    'filter': f"circle:{longitude},{latitude},{radius}",
    "limit": 1,
    "apiKey": "geoapify_key",}

    
    # Set base URL
    # base_url = "https://api.geoapify.com/v2/places"
    base_url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:{longitude},{latitude},{radius}&apiKey={geoapify_key}"



    # Make and API request using the params dictionaty
    # name_address = requests.get(base_url, params=params)
    name_address = requests.get(base_url)

    print(name_address)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
<Response [200]>
swift current - nearest hotel: Days Inn
<Response [200]>
hobyo - nearest hotel: Iftin Hotel Hobyo
<Response [200]>
sola - nearest hotel: No hotel found
<Response [200]>
mugia - nearest hotel: Parador Costa da Morte


,City,Country,Lat,Lng,Humidity,Hotel Name
180,swift current,CA,50.2834,-107.8014,27,Days Inn
334,hobyo,SO,5.3505,48.5268,85,Iftin Hotel Hobyo
411,sola,VU,-13.8833,167.5500,66,No hotel found
488,mugia,ES,43.1041,-9.2179,86,Parador Costa da Morte


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
tiles = gv.tile_sources.OSM

points = hotel_df.hvplot.points('Lng', 'Lat', size='Humidity', geo=True, frame_width= 700, frame_height= 500, tiles=tiles,
                                title='City Hotels', color = "City")
# Display the map
points


NameError: name 'gv' is not defined